# I. Load required libraries

In [8]:
import numpy as np
import pandas
import sklearn
import matplotlib.pyplot

dataframe = pandas.read_csv("faultdataset-Bus7.csv")
dataframe.sample(n=5)

,Time,Bus 1_Vm,Bus 2_Vm,Bus 3_Vm,Bus 4_Vm,Bus 5_Vm,Bus 6_Vm,Bus 7_Vm,Bus 8_Vm,Bus 9_Vm,...,Bus 4_PA,Bus 5_PA,Bus 6_PA,Bus 7_PA,Bus 8_PA,Bus 9_PA,Freq 1,Freq 2,Freq 3,Fault_duration_location
76,2.00,1.34,1.37,1.41,1.34,1.33,1.40,1.37,1.37,1.30,...,0.54,0.53,0.67,0.64,0.69,0.52,0.01,-0.08,0.00,7.35
10919,0.12,1.00,1.00,1.00,0.99,0.98,1.00,0.99,1.00,0.96,...,-0.04,-0.07,0.03,0.01,0.07,-0.08,0.00,0.00,0.00,5.35
23768,1.21,0.01,0.70,0.68,0.18,0.32,0.59,0.58,0.59,0.31,...,-1.22,-1.34,-1.35,-1.27,-1.14,-1.22,0.02,0.03,0.02,1.30
2028,1.51,1.17,1.20,1.19,1.17,1.16,1.19,1.17,1.19,1.14,...,0.18,0.13,0.18,0.13,0.17,0.11,0.00,0.01,-0.01,3.20
20472,2.97,1.06,0.97,0.97,1.02,0.99,0.99,0.97,0.98,0.97,...,0.91,0.88,0.99,0.97,1.02,0.87,-0.22,-0.23,-0.23,4.50


In [9]:
import sys
print(sys.executable)

/Users/matthew/Programs/machine_learning/venv/bin/python3


In [10]:
#convert the data into an array
dataset = dataframe.values
X = dataset[0:,1:22].astype(float)
y = dataset[0:,22].astype(float)


#Split the data into training and testing parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

NameError: name 'train_test_split' is not defined

# III. Training Models

## K-Nearest Neighbors (KNN) Model##

### Hypertuning: GridSearchCV

In [21]:
knn_param={
    'n_neighbors':[5,9,17,21,53,101],
    'weights':['uniform','distance'],
    # ERRROR: Euclidean mispelled...Geesh
    'metric': ['euclidean','manhattan']
}

In [32]:
# ERROR: return_train_score='true' should return_train_score=True
knn=RandomizedSearchCV(KNeighborsRegressor(), knn_param, verbose=1, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', return_train_score=True)
knn_results=knn.fit(X,y)
knn_results.best_score_
knn_results.best_estimator_
knn_results.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'weights': 'uniform', 'n_neighbors': 101, 'metric': 'manhattan'}

### Testing results

#### n_neighbors=70

In [61]:
#KNN
knn = KNeighborsRegressor(metric='manhattan', n_neighbors=70, weights='distance').fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
mae_knn=mean_absolute_error(y_test,knn_predictions)
mse_knn=mean_squared_error(y_test,knn_predictions)
r2_knn=r2_score(y_test,knn_predictions)
print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))



KNN model
Mean Absolute Error (MAE) 1.9454126346211391
Mean Squared Error (MSE) 5.683803259392609
R Squared 0.1445056782877493


#### n_neighbors=140

In [62]:
#KNN
knn = KNeighborsRegressor(metric='manhattan', n_neighbors=140, weights='distance').fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
mae_knn=mean_absolute_error(y_test,knn_predictions)
mse_knn=mean_squared_error(y_test,knn_predictions)
r2_knn=r2_score(y_test,knn_predictions)
print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))



KNN model
Mean Absolute Error (MAE) 1.9712956146471616
Mean Squared Error (MSE) 5.7743453766789
R Squared 0.13087778448508725


## Random Forest Regressor (RFR) Model

In [33]:
rf_param = {
    'n_estimators': [100, 1000, 10000],
    # ERROR: 'max_features':['none', 'sqrt', 'log2'] needs to be 'max_features': [None, 'sqrt', 'log2']
    'max_features': [None, 'sqrt', 'log2'],
    'max_depth' : [4,6]
}

In [31]:
# ERROR: return_train_score='true' should return_train_score=True
rf_gridsearch=GridSearchCV(RandomForestRegressor(), rf_param, verbose=1, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', return_train_score=True)
rf_results=rf_gridsearch.fit(X,y)
rf_results.best_score_
rf_results.best_estimator_
rf_results.best_params_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


{'max_depth': 4, 'max_features': None, 'n_estimators': 10000}

### Testing results

#### n_estimators = 50

In [59]:
#RF
# Error: InvalidParameterError: The 'max_features' parameter of RandomForestRegressor must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'log2', 'sqrt'} or None. Got 'auto' instead.
#rf = RandomForestRegressor(n_estimators=1000, max_features='auto')
rf = RandomForestRegressor(n_estimators=50, max_features=None)
rf.fit(X_train,y_train)
rf_prediction=rf.predict(X_test)
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)
print("\nRandom Forest Model: 1000 Estimators")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))

#learning curve
#train_sizes_rf, train_scores_rf, validation_scores_rf = learning_curve(rf,X,y, cv=5, scoring = 'neg_mean_squared_error')


Random Forest Model: 1000 Estimators
Mean Absolute Error (MAE) 1.2549365723153647
Mean Squared Error (MSE) 3.2550954851039284
R Squared 0.5100612077774171


#### n_estimators = 150

In [60]:
#RF
# Error: InvalidParameterError: The 'max_features' parameter of RandomForestRegressor must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'log2', 'sqrt'} or None. Got 'auto' instead.
#rf = RandomForestRegressor(n_estimators=1000, max_features='auto')
rf = RandomForestRegressor(n_estimators=150, max_features=None)
rf.fit(X_train,y_train)
rf_prediction=rf.predict(X_test)
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)
print("\nRandom Forest Model: 1000 Estimators")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))

#learning curve
#train_sizes_rf, train_scores_rf, validation_scores_rf = learning_curve(rf,X,y, cv=5, scoring = 'neg_mean_squared_error')


Random Forest Model: 1000 Estimators
Mean Absolute Error (MAE) 1.2479876220328199
Mean Squared Error (MSE) 3.215878617787794
R Squared 0.5159639116137706


## Naive Bayes (NB) Model

In [52]:
nb_param={
    #ValueError: Invalid parameter 'n_iter' for estimator BayesianRidge
    #'n_iter':[100, 200,300, 400, 500],
    'max_iter':[100, 200,300, 400, 500],
    'lambda_1':[ 1,1e-06, 1e-02, 1e-04],
    'alpha_1': [ 1,1e-06, 1e-02, 1e-04]
}
# ERROR: return_train_score='true' should return_train_score=True
nb=GridSearchCV(linear_model.BayesianRidge(),nb_param,verbose=1,cv=5,n_jobs=-1, scoring= 'neg_mean_squared_error',  return_train_score=True)
nb_results=nb.fit(X,y)
nb_results.best_score_
nb_results.best_estimator_
nb_results.best_params_

Fitting 5 folds for each of 80 candidates, totalling 400 fits


{'alpha_1': 1, 'lambda_1': 1e-06, 'max_iter': 100}

### Testing results

#### max_iter = 150

In [42]:
#NB
#TypeError: __init__() got an unexpected keyword argument 'n_iter'
# nb =linear_model.BayesianRidge(n_iter=300, lambda_1= 1e-06, alpha_1=1).fit(X_train, y_train)
nb =linear_model.BayesianRidge(max_iter=150, lambda_1= 1e-06, alpha_1=1).fit(X_train, y_train)
gnb_predictions = nb.predict(X_test)
mae_nb=mean_absolute_error(y_test,gnb_predictions)
mse_nb=mean_squared_error(y_test,gnb_predictions)
r2_nb=r2_score(y_test,gnb_predictions)
print("\nNaive Bayesian model")
print("Mean Absolute Error (MAE) "+str(mae_nb))
print("Mean Squared Error (MSE) "+str(mse_nb))
print("R Squared "+str(r2_nb))


Naive Bayesian model
Mean Absolute Error (MAE) 2.066736544323985
Mean Squared Error (MSE) 6.082267773215986
R Squared 0.0845310251509761


#### max_iter = 250

In [63]:
#NB
#TypeError: __init__() got an unexpected keyword argument 'n_iter'
# nb =linear_model.BayesianRidge(n_iter=300, lambda_1= 1e-06, alpha_1=1).fit(X_train, y_train)
nb =linear_model.BayesianRidge(max_iter=250, lambda_1= 1e-06, alpha_1=1).fit(X_train, y_train)
gnb_predictions = nb.predict(X_test)
mae_nb=mean_absolute_error(y_test,gnb_predictions)
mse_nb=mean_squared_error(y_test,gnb_predictions)
r2_nb=r2_score(y_test,gnb_predictions)
print("\nNaive Bayesian model")
print("Mean Absolute Error (MAE) "+str(mae_nb))
print("Mean Squared Error (MSE) "+str(mse_nb))
print("R Squared "+str(r2_nb))

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.

Naive Bayesian model
Mean Absolute Error (MAE) 2.066736544323985
Mean Squared Error (MSE) 6.082267773215986
R Squared 0.0845310251509761


## Decision Tree (DT) Model

In [46]:
# This block of code failed to terminate in a reasonable period of time. 

#dt_param={
 #   'random_state':[0,1,2, 3, 4, 5],
 #   'min_samples_leaf':[ 1,6, 12, 18],
 #   'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
#}

#dt=GridSearchCV(DecisionTreeRegressor(), dt_param, verbose=1, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', return_train_score=True)
#dt_results=dt.fit(X,y)
#dt_results.best_score_
#dt_results.best_estimator_
#dt_results.best_params_

### Testing resutls

#### min_samples_leaf = 15

In [64]:
#DT
dt = DecisionTreeRegressor(min_samples_leaf=15,  max_features=20, random_state= 4 ).fit(X_train, y_train)
dtree_prediction = dt.predict(X_test)
mae_dt=mean_absolute_error(y_test,dtree_prediction)
mse_dt=mean_squared_error(y_test,dtree_prediction)
r2_dt=r2_score(y_test,dtree_prediction)

print("\nDecision tree model")
print("Mean Absolute Error (MAE) "+str(mae_dt))
print("Mean Squared Error (MSE) "+str(mse_dt))
print("R Squared "+str(r2_dt))


Decision tree model
Mean Absolute Error (MAE) 1.490420772173397
Mean Squared Error (MSE) 4.223145621776413
R Squared 0.36435570791031247


#### min_samples_leaf = 25

In [65]:
#DT
dt = DecisionTreeRegressor(min_samples_leaf=25,  max_features=20, random_state= 4 ).fit(X_train, y_train)
dtree_prediction = dt.predict(X_test)
mae_dt=mean_absolute_error(y_test,dtree_prediction)
mse_dt=mean_squared_error(y_test,dtree_prediction)
r2_dt=r2_score(y_test,dtree_prediction)

print("\nDecision tree model")
print("Mean Absolute Error (MAE) "+str(mae_dt))
print("Mean Squared Error (MSE) "+str(mse_dt))
print("R Squared "+str(r2_dt))


Decision tree model
Mean Absolute Error (MAE) 1.5466888765266227
Mean Squared Error (MSE) 4.3256465132523285
R Squared 0.34892784620817097


## Neural Network (NN) Model

### Testing results

#### hidden_layer_size=50

In [69]:
NN1 = MLPRegressor(max_iter=1500, activation='relu', solver='adam', learning_rate_init=0.001, hidden_layer_sizes=50).fit(X_train,y_train)
NN1_prediction = NN1.predict(X_test)
mae_NN1=mean_absolute_error(y_test,NN1_prediction)
mse_NN1=mean_squared_error(y_test,NN1_prediction)
r2_NN1=r2_score(y_test,NN1_prediction)
print("NN with Relu activation function")
print("Mean Absolute Error (MAE) "+str(mae_NN1))
print("Mean Squared Error (MSE) "+str(mse_NN1))
print("R Squared "+str(r2_NN1))

NN with Relu activation function
Mean Absolute Error (MAE) 1.8115107105044252
Mean Squared Error (MSE) 4.977857471995471
R Squared 0.2507606953939313


#### hidden_layer_size=100

In [70]:
NN1 = MLPRegressor(max_iter=1500, activation='relu', solver='adam', learning_rate_init=0.001, hidden_layer_sizes=100).fit(X_train,y_train)
NN1_prediction = NN1.predict(X_test)
mae_NN1=mean_absolute_error(y_test,NN1_prediction)
mse_NN1=mean_squared_error(y_test,NN1_prediction)
r2_NN1=r2_score(y_test,NN1_prediction)
print("NN with Relu activation function")
print("Mean Absolute Error (MAE) "+str(mae_NN1))
print("Mean Squared Error (MSE) "+str(mse_NN1))
print("R Squared "+str(r2_NN1))

NN with Relu activation function
Mean Absolute Error (MAE) 1.7318143899701244
Mean Squared Error (MSE) 4.635700247889923
R Squared 0.3022602897670137


## Support Vector Machine (SVM) Model

### Testing results

#### kernel = 'poly'

In [74]:
SVM_model_2 = SVR(kernel = 'poly', C = 10, gamma = 'scale').fit(X_train, y_train)
SVM_predictions_2 = SVM_model_2.predict(X_test)
mae_SVM_2=mean_absolute_error(y_test,SVM_predictions_2)
mse_SVM_2=mean_squared_error(y_test,SVM_predictions_2)
r2_SVM_2=r2_score(y_test,SVM_predictions_2)
print("\nSVR with a rbf kernel")
print("Mean Absolute Error (MAE) "+str(mae_SVM_2))
print("Mean Squared Error (MSE) "+str(mse_SVM_2))
print("R Squared "+str(r2_SVM_2))


SVR with a rbf kernel
Mean Absolute Error (MAE) 1.9964323024698485
Mean Squared Error (MSE) 6.015470472991519
R Squared 0.0945849817078479


#### kernel = 'linear'

In [75]:
SVM_model_2 = SVR(kernel = 'linear', C = 10, gamma = 'scale').fit(X_train, y_train)
SVM_predictions_2 = SVM_model_2.predict(X_test)
mae_SVM_2=mean_absolute_error(y_test,SVM_predictions_2)
mse_SVM_2=mean_squared_error(y_test,SVM_predictions_2)
r2_SVM_2=r2_score(y_test,SVM_predictions_2)
print("\nSVR with a rbf kernel")
print("Mean Absolute Error (MAE) "+str(mae_SVM_2))
print("Mean Squared Error (MSE) "+str(mse_SVM_2))
print("R Squared "+str(r2_SVM_2))


SVR with a rbf kernel
Mean Absolute Error (MAE) 2.0486280687371323
Mean Squared Error (MSE) 6.170430553698182
R Squared 0.07126125583511456


## IV. Simulate Missing Data

#### Generate missing data by percentage 10%

In [80]:
#Generate missing data by percentage 10%
import warnings

def ampute_mcar(X_complete, missing_rate = 0.1):
    # Mask completely at random some values

    M = np.random.binomial(1, missing_rate, size = X_complete.shape)
    X_obs = X_complete.copy()
    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated missing values: {}'.\
      format(np.round(np.sum(np.isnan(X_obs))/X_obs.size,3)))

    # warning if a full row is missing
    for row in X_obs:
        if np.all(np.isnan(row)):
            warnings.warn('Some row(s) contains only nan values.')
            break

    # warning if a full col is missing
    for col in X_obs.T:
        if np.all(np.isnan(col)):
            warnings.warn('Some col(s) contains only nan values.')
            break

    return X_obs

X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.1)

#Convert NAN values into 0
from numpy import *
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

Percentage of newly generated missing values: 0.1


#### Re-Training RFR including missing data

In [86]:
# RFR: Generate 20% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.2)

#Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(X_train_missing_data,y_train)

# Test
rf_prediction=rf.predict(X_test)

# Output metrics
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)

print("\nRFR model")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))


Percentage of newly generated missing values: 0.2

RFR model
Mean Absolute Error (MAE) 1.5282211605494629
Mean Squared Error (MSE) 3.9866655014608745
R Squared 0.3999493748433569


In [87]:
# RFR: Generate 40% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.4)

# Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(X_train_missing_data,y_train)

# Test
rf_prediction=rf.predict(X_test)

# Output metrics
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)

print("\nRFR model")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))


Percentage of newly generated missing values: 0.4

RFR model
Mean Absolute Error (MAE) 1.7067877644459155
Mean Squared Error (MSE) 4.548666142487701
R Squared 0.31536017721365683


In [88]:
# RFR: Generate 60% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.6)

# Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(X_train_missing_data,y_train)

# Test
rf_prediction=rf.predict(X_test)

# Output metrics
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)

print("\nRFR model")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))


Percentage of newly generated missing values: 0.6


/var/folders/_f/4gpz1vkd4x12810cr6067xc40000gn/T/ipykernel_33960/3871708711.py:16: UserWarning: Some row(s) contains only nan values.
  warnings.warn('Some row(s) contains only nan values.')



RFR model
Mean Absolute Error (MAE) 1.9070431098171805
Mean Squared Error (MSE) 5.31928404324804
R Squared 0.19937107480747396


In [89]:
# RFR: Generate 80% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.8)

# Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(X_train_missing_data,y_train)

# Test
rf_prediction=rf.predict(X_test)

# Output metrics
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)

print("\nRFR model")
print("Mean Absolute Error (MAE) "+str(mae_rf))
print("Mean Squared Error (MSE) "+str(mse_rf))
print("R Squared "+str(r2_rf))


Percentage of newly generated missing values: 0.799


/var/folders/_f/4gpz1vkd4x12810cr6067xc40000gn/T/ipykernel_33960/3871708711.py:16: UserWarning: Some row(s) contains only nan values.
  warnings.warn('Some row(s) contains only nan values.')



RFR model
Mean Absolute Error (MAE) 2.0712628695157007
Mean Squared Error (MSE) 5.907038613704783
R Squared 0.11090553955953153


#### Re-Training KNN model including missing data

In [90]:
# KNN: Generate 20% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.2)

#Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
knn = KNeighborsRegressor()
knn.fit(X_train_missing_data,y_train)

# Test
knn_prediction=knn.predict(X_test)

# Output metrics
mae_knn=mean_absolute_error(y_test,knn_prediction)
mse_knn=mean_squared_error(y_test,knn_prediction)
r2_knn=r2_score(y_test,knn_prediction)

print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))


Percentage of newly generated missing values: 0.199

KNN model
Mean Absolute Error (MAE) 2.176929626846982
Mean Squared Error (MSE) 6.949971475081393
R Squared -0.04607088979881779


In [91]:
# KNN: Generate 40% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.4)

#Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
knn = KNeighborsRegressor()
knn.fit(X_train_missing_data,y_train)

# Test
knn_prediction=knn.predict(X_test)

# Output metrics
mae_knn=mean_absolute_error(y_test,knn_prediction)
mse_knn=mean_squared_error(y_test,knn_prediction)
r2_knn=r2_score(y_test,knn_prediction)

print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))




Percentage of newly generated missing values: 0.401

KNN model
Mean Absolute Error (MAE) 2.3016053092912596
Mean Squared Error (MSE) 7.609781943400952
R Squared -0.1453818763501018


In [92]:
# KNN: Generate 60% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.6)

#Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
knn = KNeighborsRegressor()
knn.fit(X_train_missing_data,y_train)

# Test
knn_prediction=knn.predict(X_test)

# Output metrics
mae_knn=mean_absolute_error(y_test,knn_prediction)
mse_knn=mean_squared_error(y_test,knn_prediction)
r2_knn=r2_score(y_test,knn_prediction)

print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))


Percentage of newly generated missing values: 0.6


/var/folders/_f/4gpz1vkd4x12810cr6067xc40000gn/T/ipykernel_33960/3871708711.py:16: UserWarning: Some row(s) contains only nan values.
  warnings.warn('Some row(s) contains only nan values.')



KNN model
Mean Absolute Error (MAE) 2.340160280490859
Mean Squared Error (MSE) 7.891099599298773
R Squared -0.18772423871464738


In [93]:
# KNN: Generate 80% of missing data
X_train_missing_data = ampute_mcar(X_train, missing_rate = 0.8)

#Convert NAN values into 0
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

# Train
knn = KNeighborsRegressor()
knn.fit(X_train_missing_data,y_train)

# Test
knn_prediction=knn.predict(X_test)

# Output metrics
mae_knn=mean_absolute_error(y_test,knn_prediction)
mse_knn=mean_squared_error(y_test,knn_prediction)
r2_knn=r2_score(y_test,knn_prediction)

print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn))
print("Mean Squared Error (MSE) "+str(mse_knn))
print("R Squared "+str(r2_knn))


Percentage of newly generated missing values: 0.8


/var/folders/_f/4gpz1vkd4x12810cr6067xc40000gn/T/ipykernel_33960/3871708711.py:16: UserWarning: Some row(s) contains only nan values.
  warnings.warn('Some row(s) contains only nan values.')



KNN model
Mean Absolute Error (MAE) 2.477389181066867
Mean Squared Error (MSE) 8.76106076884548
R Squared -0.31866593509153596


# End